In [11]:
from langchain import hub
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter,TokenTextSplitter,CharacterTextSplitter
from langchain_cohere import CohereEmbeddings

In [12]:
from dotenv import load_dotenv
load_dotenv('/Users/eshantdas/Desktop/YOutube_scripts/myenv/.env')

True

In [13]:
loader = PyMuPDFLoader("/Users/eshantdas/Downloads/field-guide-to-data-science.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=600)
splits_character = text_splitter.split_documents(data)

embeddings = CohereEmbeddings(
    model="embed-english-light-v3.0"
)  

vectorstore = Chroma.from_documents(documents=splits_character, embedding=embeddings)
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

In [27]:
prompt.messages[0]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [31]:
rag_chain.invoke("Tell me about Data Science?")

'Data Science is the art of turning data into actions through the creation of data products, which provide actionable information without exposing decision makers to the underlying data or analytics. It involves extracting timely, actionable information from diverse data sources to drive data products, such as movie recommendations, weather forecasts, and targeted advertising.'